In [4]:
import pandas as pd
import lib.strat2file as strat

r_arrow = u'\N{BLACK RIGHT-POINTING TRIANGLE}'

#Read stratigraphic chart and make strat chart easily searchable
chart = pd.read_excel('ChronostratChart2018-08.xlsx')
chart = chart.applymap(lambda s:s.strip().lower() if type(s) == str else s)

#Have a look?
#chart

Run a few tests:

In [7]:
tests = ['Sandbian', 
         'Phanerozoic', 
         'Meghalayan', 
         'upper Cretaceous', 
         'Lower ordovician'] 
    
for test in tests: 
    print(test, r_arrow, strat.geo_to_year(test, chart) )

Sandbian ▶ (453.0, 458.4)
Phanerozoic ▶ (0.0, 541.0)
Meghalayan ▶ (0.0, 0.0042)
upper Cretaceous ▶ (66.0, 100.5)
Lower ordovician ▶ (470.0, 485.4)


In [ ]:
for test in tests: 
    print(test, arrow, get_time(test, uncertainty='max'), get_time(test, uncertainty='min')  )

In [ ]:
#upper case
print(get_time('danian'))
#mix
print(get_time('ARcheAN'))

In [ ]:
print(get_time('latest Cretaceous'))
print(get_time('late Cretaceous'))
print(get_time('early Cretaceous'))
print(get_time('earliest Cretaceous'))
print(get_time('Cretaceous'))
print(get_time('mid Cretaceous'))

In [ ]:
chart

